In [ ]:
import pandas as pd
import os
from typing import List
import datetime

In [ ]:
# Specify the directory path as a string
directory_path: str = 'data'

#index/time/divice/mac_id/rssi/sward_id/zone_id
df: pd.DataFrame = pd.read_csv('data/tjlabs_sward.csv')


df.columns = ['index', 'time', 'device', 'mac_id', 'rssi', 'sward_id', 'zone_id']

df.head()

In [ ]:
df['time'] = pd.to_datetime(df['time']) 

df.describe()

In [ ]:
mac_counts = df.groupby('mac_id').size()

# Convert the Series to a dictionary (optional)
mac_counts_dict = mac_counts.to_dict()
mac_counts_dict.__len__()
mac_counts_dict

In [ ]:
df= df.sort_values(by=['time'])
df

In [ ]:
grouped_df = df.groupby('time')
grouped_df

In [ ]:
for _, group_df in zip(range(36), grouped_df):
    # Extract the DataFrame from the tuple
    df = group_df[1]

    # Now you can access the 'mac_id' column
    unique_mac_ids = df['mac_id'].unique()
    print(unique_mac_ids)

In [ ]:
from circular_queue import CircularQueue
queue = CircularQueue(6)

for _, group_df in zip(range(10000), grouped_df):
    # Extract the DataFrame from the tuple
    time = group_df[0]
    df = group_df[1]
    unique_mac_ids = df['mac_id'].unique()
    queue.enqueue([time, unique_mac_ids])
    if queue.new_mac_set.__len__ == 0:
        continue
    for new_mac_id in queue.new_mac_set:
        filtered_new_mac = df[(df['mac_id'] == new_mac_id) & (df['time'] ==queue.last_updated)]
        print(filtered_new_mac)       
        if queue.disappeared_mac_set.__len__ == 0:
            continue
        for dis_mac_id in queue.disappeared_mac_set:
            filtered_disappeared_mac = df[(df['mac_id'] == dis_mac_id) & (df['time'] < queue.last_updated) & (df['time'] > queue.last_updated - datetime.timedelta(minutes=5))]
            
            if filtered_disappeared_mac.empty:
                continue
            else:
                print(filtered_disappeared_mac)
            
        



In [ ]:
# MAC-Link 설계 파라미터
# 기존 MAC의 사라짐을 판단할 time window => 1분
# 기존 MAC의 사라짐을 판단할 조건 => set을 통해 사라지거나 새로 생성된 MAC을 판단
# MAC-Link 조건 (기존 MAC과 신규 MAC이 동일 MAC임을 판단하기 위한 조건)
# - 새로운 맥과 사라진 맥의 time window가 1분 이내
# - 새로운 맥과 사라진 맥의 마지막 존 아이디가 같음
# - 새로운 맥과 사라진 맥의 RSSI가 10 이내
# 동일 MAC이라고 판단 시, MAC 관리 방안
# - 기존의 uuid에 붙임

In [ ]:
cq = CircularQueue(3)

list1 = [1]
list2 = [2,3]
list3 = [4]
list4 = [4]
list5 = []

list_all = [list1, list2, list3, list4, list5]

for list in list_all:
    cq.enqueue([0, list])
    print('new', cq.new_mac_set)
    print('dis', cq.disappeared_mac_set)
    print('all', cq.mac_set)